In [1]:
import numpy as np
import pandas as pd

In [2]:
stock = 'sp500'
df = pd.read_csv(f"{stock}_output/3return.csv")
df.columns = ['index', 'daily_return']
pred = df[['daily_return']]
pred

,daily_return
0,0.996043
1,1.011158
2,1.000654
3,1.024586
4,1.024829
...,...
244,1.131925
245,1.145605
246,1.149507
247,1.156335


In [3]:
pred.to_csv(f'{stock}.csv', index=None)

In [4]:
pred['daily_return'] = pred['daily_return'] / pred['daily_return'].shift(1)
pred['daily_return'] = pred['daily_return'] - 1
pred.dropna(inplace=True)

C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_14528\490373256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred['daily_return'] = pred['daily_return'] / pred['daily_return'].shift(1)
C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_14528\490373256.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred['daily_return'] = pred['daily_return'] - 1
C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_14528\490373256.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

In [5]:
pred

,daily_return
1,0.015175
2,-0.010388
3,0.023916
4,0.000238
5,0.007287
...,...
244,-0.016803
245,0.012085
246,0.003406
247,0.005940


In [6]:
portfolio_df_performance = pred

In [7]:
alpha_df_performance = pd.DataFrame()
alpha_df_performance['portfolio_daily_return'] = portfolio_df_performance['daily_return']
alpha_df_performance['portfolio_net_value'] = (alpha_df_performance['portfolio_daily_return'] + 1).cumprod()

net_value_columns = ['portfolio_net_value']

alpha_statistics_df = pd.DataFrame(index=alpha_df_performance[net_value_columns].columns,
                                    columns=["年化收益", "年化波动率", "最大回撤率", "夏普率", "Calmar", "IR"])

# alpha_df_performance.set_index("dt", inplace=True)
alpha_df_performance.index = pd.to_datetime(alpha_df_performance.index)
monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
monthly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    monthly_statistics_df])
monthly_statistics_df = monthly_statistics_df.pct_change()
monthly_statistics_df = monthly_statistics_df.dropna()
monthly_statistics_df.index = monthly_statistics_df.index.date
## TODO 补充第一年的数据
yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
yearly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    yearly_statistics_df])
yearly_statistics_df = yearly_statistics_df.pct_change()
yearly_statistics_df = yearly_statistics_df.dropna()
yearly_statistics_df.index = yearly_statistics_df.index.date

alpha_statistics_df.loc[:, "年化收益"] = np.mean(
    (alpha_df_performance[net_value_columns].tail(1)) ** (252 / len(alpha_df_performance)) - 1)
alpha_statistics_df.loc[:, "年化波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252)
alpha_statistics_df.loc[:, "累积收益"] = np.mean(alpha_df_performance[net_value_columns].tail(1) - 1)
alpha_statistics_df.loc[:, "累积波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)
alpha_statistics_df.loc[:, "最大回撤率"] = np.min(
    (alpha_df_performance[net_value_columns] - alpha_df_performance[net_value_columns].cummax()) /
    alpha_df_performance[net_value_columns].cummax())
alpha_statistics_df.loc[:, "夏普率"] = alpha_statistics_df["年化收益"] / alpha_statistics_df["年化波动率"]
alpha_statistics_df.loc[:, "Calmar"] = alpha_statistics_df["年化收益"] / abs(alpha_statistics_df["最大回撤率"])
alpha_statistics_df.loc[:, "IR"] = np.mean(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252) / np.std(alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)

C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_14528\980728215.py:12: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_14528\980728215.py:20: FutureWarning: 'y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFra

In [8]:
alpha_statistics_df

,年化收益,年化波动率,最大回撤率,夏普率,Calmar,IR,累积收益,累积波动率
portfolio_net_value,0.16505,0.141625,-0.125878,1.165406,1.311196,1.04555,0.162229,0.008922


DeepPocket
ASR = ARR / AVol;   CR = ARR / abs(MDD)
        ARR    AVol    MDD    ASR    CR    IR
hs300 -0.036  0.135  -0.175 -0.270 -0.207 -0.258

zz500  0.006  0.127  -0.148  0.050  0.043  0.115

sp500  0.165  0.142  -0.126  1.165  1.311  1.045
   
nas100 0.346  0.157  -0.116  2.197  2.971  1.882